In [71]:
import pandas as pd
import mysql.connector as mc

In [72]:
db = mc.connect(
  host='172.18.55.6',
  user='comandato',
  passwd='comandato123',
)

In [73]:
Anio='2019'
Mes='01'
Dia_act='24'

In [74]:
fecha_ini='2018-10-01 00:00:00'
fecha_fin='2019-01-30 21:00:00'

In [75]:
fecha_inivr='2019-01-01 00:00:00'
fecha_finivr='2019-01-30 21:00:00'

# CONSULTA DE GESTION MASIVOS ASESOR

In [76]:
querr_ultmes_asesor="""SELECT * 
                       FROM sistemecuador_atm.gestion 
                       where FECHA_GESTION between '{0}' and '{1}';""".format(fecha_ini,fecha_fin)

In [77]:
querr_ultmes_masivo="""SELECT * 
                       FROM sistemecuador_atm.gestion_ivr
                       where FECHA_GESTION between '{0}' and '{1}';""".format(fecha_inivr,fecha_finivr)

In [78]:
ultmes_asesor=pd.read_sql(querr_ultmes_asesor, con=db)

In [79]:
ultmes_masivo=pd.read_sql(querr_ultmes_masivo, con=db)

### ORDEN POR RESPUESTAS

In [80]:
ultmes_asesor.RESPUESTA_OBTENIDA = ultmes_asesor.RESPUESTA_OBTENIDA.astype('category')
order_resp_ase=['ACUERDO DE PAGO','ACUERDO DE CONVENIO','LOCALIZADO SIN ACUERDO',
                'YA PAGO','VOLVER A LLAMAR','INFRACTOR CUELGA LLAMADA','RECLAMO VENDIO VEHICULO',
                'RENUENTE','ACTUALIZACION DATOS','RECLAMO','MENSAJE CON TERCERO','FALLECIDO / OTROS',
                'AQUI NO VIVE / TRABAJA','BUZON DE MENSAJES','TELEFONO DANADO','TELEFONO OCUPADO','NO CONTESTA','','SIN GESTION']
ultmes_asesor.RESPUESTA_OBTENIDA=ultmes_asesor.RESPUESTA_OBTENIDA.cat.set_categories(order_resp_ase)

ultmes_asesor.CONTACTO_GESTIONO  = ultmes_asesor.CONTACTO_GESTIONO.astype('category')
order_cont_ase=['CLIENTE','TERC. VALIDO','TERC. NO VALIDO','NO CONTACTO','SIN GESTION']
ultmes_asesor.CONTACTO_GESTIONO=ultmes_asesor.CONTACTO_GESTIONO.cat.set_categories(order_cont_ase)

ultmes_asesor=ultmes_asesor.sort_values(by=['CONTACTO_GESTIONO','RESPUESTA_OBTENIDA'],ascending = True)

In [81]:
gest_ase_porID = ultmes_asesor.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')

In [82]:
gest_ase_porID_DEM = ultmes_asesor.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE','NUMERO_GESTION'], keep='first')

In [83]:
ultmes_masivo.RESPUESTA_OBTENIDA=ultmes_masivo.RESPUESTA_OBTENIDA.astype('category')
order_resp_ivr=['CONTESTA IVR','SMS ENVIADO','NO CONTESTA IVR','','SIN GESTION']
ultmes_masivo.RESPUESTA_OBTENIDA=ultmes_masivo.RESPUESTA_OBTENIDA.cat.set_categories(order_resp_ivr)

ultmes_masivo = ultmes_masivo.sort_values(by=['RESPUESTA_OBTENIDA'],ascending=True)

In [84]:
gest_mas_porID = ultmes_masivo.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'],keep='first')

In [85]:
gest_mas_porID_DEM = ultmes_masivo.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE','NUMERO_GESTION'],keep='first')

In [86]:
campos=['NRO_IDENTIFICACION_CLIENTE','RESPUESTA_OBTENIDA','CONTACTO_GESTIONO']
campos1=['NRO_IDENTIFICACION_CLIENTE','RESPUESTA_OBTENIDA','CONTACTO_GESTIONO','NUMERO_GESTION']

In [87]:
gest_ase_porID = gest_ase_porID[campos]
gest_ase_porID = gest_ase_porID.rename(columns={'RESPUESTA_OBTENIDA':'CALL_MES','CONTACTO_GESTIONO':'CONT_MES'})

gest_ase_porID_DEM = gest_ase_porID_DEM[campos1]
gest_ase_porID_DEM = gest_ase_porID_DEM.rename(columns={'RESPUESTA_OBTENIDA':'CALL_MESTEL','CONTACTO_GESTIONO':'CONT_MESTEL','NUMERO_GESTION':'NUMERO'})

gest_mas_porID = gest_mas_porID[campos]
gest_mas_porID = gest_mas_porID.rename(columns={'RESPUESTA_OBTENIDA':'CALL_MESMAS','CONTACTO_GESTIONO':'CONT_MESMAS'})

gest_mas_porID_DEM = gest_mas_porID_DEM[campos1]
gest_mas_porID_DEM = gest_mas_porID_DEM.rename(columns={'RESPUESTA_OBTENIDA':'CALL_MESMASTEL','CONTACTO_GESTIONO':'CONT_MESMASTEL','NUMERO_GESTION':'NUMERO'})

In [88]:
gest_ase_porID_DEM.head()

,NRO_IDENTIFICACION_CLIENTE,CALL_MESTEL,CONT_MESTEL,NUMERO
4,0912518727,ACUERDO DE PAGO,CLIENTE,0959735900
13,0912986692,ACUERDO DE PAGO,CLIENTE,0979748680
17,0925191447,ACUERDO DE PAGO,CLIENTE,042603956
51,1802949220,ACUERDO DE PAGO,CLIENTE,0997490924
60,1301207807,ACUERDO DE PAGO,CLIENTE,0939006457


### ASIGNACION

In [89]:
def agregar_ceros(base):
    ced=base['IDENTIFICACION_DEUDOR']
    if len(ced)==9 or len(ced)==12 :
        if ced.isdigit():
            return '0'+ced
    return ced

In [90]:
def verificar_tipo(base):
    ced=base['ID']
    if len(ced)==13:
        return 'RUC'
    return 'CED'

In [91]:
def ultimo_digito(base):
    placa=base['PLACA_VEHICULO']
    placa=str(placa)
    if (placa == ''):
        return '10'
    if placa[-1].isdigit():
        return placa[-1]
    return '10'

In [92]:
def convenio(base):
    if (base['SALDO'] >= 115.8 and base['DIAS_MORAmin'] >= 180):
        return 'SI'
    return 'NO'

In [93]:
def diasmora(base):
    if(base['DIAS_MORAmin'] >= 180):
        return 'MAYOR A 180'
    return 'MENOR A 180'    

In [94]:
#nombre de los archivos Asignacion normal
nom_actual_cor='..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_act+'_'+Mes+'_'+Anio+'.xlsx'
nombre_hoja_cor='Asignacion'
cartera_cor='CORRIENTE'

# nombre de los archivos Menores
nom_actual_men='..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_act+'_'+Mes+'_'+Anio+'.xlsx'
nombre_hoja_men='Menores'
cartera_men='MENORES'

# nombre de los archivos Demografico
nom_actual_dem='..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_act+'_'+Mes+'_'+Anio+'.xlsx'
nombre_hoja_dem='Demografico'

In [95]:
#leer archivo excel asignacion actual y anterior
asignacion_actual_cor=pd.read_excel(open(nom_actual_cor,'rb'), sheet_name=nombre_hoja_cor, dtype={"IDENTIFICACION_DEUDOR": str})
#leer archivo excel asignacion actual y anterior
asignacion_actual_men=pd.read_excel(open(nom_actual_men,'rb'), sheet_name=nombre_hoja_men, dtype={"IDENTIFICACION_DEUDOR": str})
#leer archivo excel asignacion actual y anterior
asignacion_actual_dem=pd.read_excel(open(nom_actual_dem,'rb'), sheet_name=nombre_hoja_dem, dtype={"IDENTIFICACION_DEUDOR": str,'TELEFONO':str})

In [96]:
asignacion_actual_cor= asignacion_actual_cor.fillna('')
asignacion_actual_cor['ID']=asignacion_actual_cor.apply(agregar_ceros,axis=1)
asignacion_actual_cor['SUBCAMPANAPORCLIENTE'].replace(to_replace=[''],value='4. VENCIDA FASE I',inplace=True)
try:
    asignacion_actual_cor['FECHA_COACTIVA'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
except:
    asignacion_actual_cor['FECHA_COACTIVO'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
asignacion_actual_cor['NOMBRE']=asignacion_actual_cor['RAZONSOCIAL']+' '+asignacion_actual_cor['PRIMER_APELLIDO']+' '+asignacion_actual_cor['SEGUNDO_APELLIDO']+' '+asignacion_actual_cor['PRIMER_NOMBRE']
asignacion_actual_cor['CARTERA']='CORRIENTE'

In [97]:
asignacion_actual_cor['TIPO']=asignacion_actual_cor.apply(verificar_tipo,axis=1)
asignacion_actual_cor['U_PLACA']=asignacion_actual_cor.apply(ultimo_digito,axis=1)

In [98]:
try:
    base_gestion_cor=asignacion_actual_cor[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVA','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA']]
    ordenada_cor=base_gestion_cor.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO']).agg({'FECHA_COACTIVA':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})
except:
    base_gestion_cor=asignacion_actual_cor[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVO','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA']]
    ordenada_cor=base_gestion_cor.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO']).agg({'FECHA_COACTIVO':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})

In [99]:
asignacion_actual_men= asignacion_actual_men.fillna('')
asignacion_actual_men['ID']=asignacion_actual_men.apply(agregar_ceros,axis=1)
asignacion_actual_men['SUBCAMPANAPORCLIENTE'].replace(to_replace=[''],value='4. VENCIDA FASE I',inplace=True)
try:
    asignacion_actual_men['FECHA_COACTIVA'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
except:
    asignacion_actual_men['FECHA_COACTIVO'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
asignacion_actual_men['NOMBRE']=asignacion_actual_men['RAZONSOCIAL']+' '+asignacion_actual_men['PRIMER_APELLIDO']+' '+asignacion_actual_men['SEGUNDO_APELLIDO']+' '+asignacion_actual_men['PRIMER_NOMBRE']
asignacion_actual_men['CARTERA']='MENORES'

In [100]:
asignacion_actual_men['TIPO']=asignacion_actual_men.apply(verificar_tipo,axis=1)
asignacion_actual_men['U_PLACA']=asignacion_actual_men.apply(ultimo_digito,axis=1)

In [101]:
try:
    base_gestion_men=asignacion_actual_men[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVA','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA']]
    ordenada_men=base_gestion_men.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO']).agg({'FECHA_COACTIVA':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})
except:
    base_gestion_men=asignacion_actual_men[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVO','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA']]
    ordenada_men=base_gestion_men.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO']).agg({'FECHA_COACTIVO':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})

In [102]:
base_total=pd.concat([ordenada_cor, ordenada_men])

In [103]:
base_total = base_total.reset_index()

In [104]:
base_total.columns = ['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO','FECHA_COACTIVA','SALDO','DIAS_MORAmin','DIAS_MORAmax','U_PLACA']

In [105]:
base_total.drop_duplicates(['IDENTIFICACION_DEUDOR'],keep='first',inplace=True)

In [106]:
base_total['GENERACION_CONVENIO']=base_total.apply(convenio,axis=1)
base_total['MORA']=base_total.apply(diasmora,axis=1)

In [107]:
base_total = base_total.rename(columns={'ID':'NRO_IDENTIFICACION_CLIENTE'})

In [108]:
base_total.head()

,IDENTIFICACION_DEUDOR,NRO_IDENTIFICACION_CLIENTE,NOMBRE,SUBCAMPANAPORCLIENTE,CARTERA,TIPO,FECHA_COACTIVA,SALDO,DIAS_MORAmin,DIAS_MORAmax,U_PLACA,GENERACION_CONVENIO,MORA
0,00042448K,00042448K,ARAGON JUAREZ SERGIO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,94,94,9,NO,MENOR A 180
1,1000559797,1000559797,RECALDE ESPINOSA ALFONSO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,189.7,36,451,10,NO,MENOR A 180
2,1000580975,1000580975,VACA JERVIS SONIA,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,135,135,0,NO,MENOR A 180
3,1001296837,1001296837,BENITEZ POZO IGNACIO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,140,316,9,NO,MENOR A 180
4,1001342284,1001342284,PORTILLA RAMIREZ SEGUNDO,4. VENCIDA FASE I,CORRIENTE,CED,NO COACTIVA,115.8,38,38,3,NO,MENOR A 180


In [109]:
especiales = pd.read_excel(open('DEMOGRAFICOS.xlsx','rb'), sheet_name='BASE ESPECIAL', dtype={"IDENTIFICACION_DEUDOR": str})

In [110]:
base_total1 = pd.merge(base_total,especiales,on=['IDENTIFICACION_DEUDOR'],how='left')

### CRUCES CON LA ASIGNACION Y LAS GESTIONES

In [111]:
primercruce = pd.merge(base_total1,gest_ase_porID,on=['NRO_IDENTIFICACION_CLIENTE'],how='left')

In [112]:
segundocruce = pd.merge(primercruce,gest_mas_porID,on=['NRO_IDENTIFICACION_CLIENTE'],how='left')

In [113]:
segundocruce.head()

,IDENTIFICACION_DEUDOR,NRO_IDENTIFICACION_CLIENTE,NOMBRE,SUBCAMPANAPORCLIENTE,CARTERA,TIPO,FECHA_COACTIVA,SALDO,DIAS_MORAmin,DIAS_MORAmax,...,GENERACION_CONVENIO,MORA,PRIORIDAD,CAMPAÑA,BASE_FOCALIZADA,CUOTA_VENCIDA,CALL_MES,CONT_MES,CALL_MESMAS,CONT_MESMAS
0,00042448K,00042448K,ARAGON JUAREZ SERGIO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,94,94,...,NO,MENOR A 180,NaN,MENOR A 180,NO,SI,NaN,NaN,NaN,NaN
1,1000559797,1000559797,RECALDE ESPINOSA ALFONSO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,189.7,36,451,...,NO,MENOR A 180,1.0,ACUERDOS INCUMPLIDOS,NO,NO,ACUERDO DE PAGO,CLIENTE,CONTESTA IVR,TITULAR
2,1000580975,1000580975,VACA JERVIS SONIA,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,135,135,...,NO,MENOR A 180,NaN,MENOR A 180,NO,NO,NaN,NaN,NaN,NaN
3,1001296837,1001296837,BENITEZ POZO IGNACIO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,140,316,...,NO,MENOR A 180,2.0,SIN CLASE,BASE FOCALIZADA,NO,NO CONTESTA,NO CONTACTO,CONTESTA IVR,TITULAR
4,1001342284,1001342284,PORTILLA RAMIREZ SEGUNDO,4. VENCIDA FASE I,CORRIENTE,CED,NO COACTIVA,115.8,38,38,...,NO,MENOR A 180,1.0,ACUERDOS INCUMPLIDOS,NO,SI,ACUERDO DE PAGO,CLIENTE,CONTESTA IVR,TITULAR


In [114]:
segundocruce.NRO_IDENTIFICACION_CLIENTE.count()

34102

## Consultar numeros de la BD

### -Querry para la consulta de telefonos

In [115]:
consulta="""SELECT 
                CONCAT(CEDULA) as IDENTIFICACION_DEUDOR,
                CONCAT('0',NUMERO) as NUMERO,
                CASE CONTACTO
                    WHEN '1' THEN 1
                    WHEN '2' THEN 2
                    WHEN '3' THEN 5
                    WHEN '4' THEN 4
                    WHEN '5' THEN 3
                    WHEN ' ' THEN 3
                    WHEN '' THEN 3
                    WHEN 'TITULAR' THEN 1
                    WHEN 'TERCERO' THEN 2
                    WHEN 'EQUIVOCADO' THEN 5
                    WHEN 'NO CONTACTO' THEN 4
                    WHEN NULL THEN 3
                    ELSE  3
                END  AS  dueno
            FROM
                TELEFONICOS_NUMEROS.TELEFONOS  HAVING dueno IN (1,2,5)  ORDER BY dueno;"""

In [116]:
DEMO=pd.read_sql(consulta, con=db)

In [117]:
DEMO.count()

IDENTIFICACION_DEUDOR    260239
NUMERO                   260239
dueno                    260239
dtype: int64

In [118]:
leer_demografico=pd.read_excel(open('DEMOGRAFICOS.xlsx','rb'), sheet_name='DEMOGRAFICOS', dtype={"IDENTIFICACION_DEUDOR": str,"NUMERO":str,'DEMO':str})

In [119]:
cruce_num = pd.merge(segundocruce,leer_demografico,on='IDENTIFICACION_DEUDOR',how='left')

In [120]:
cruce_num2 = pd.merge(cruce_num,DEMO,on=['IDENTIFICACION_DEUDOR','NUMERO'],how='left')
cruce_num2['dueno'].replace(to_replace=[1,2,5],value=['TITULAR','TERCERO','EQUIVOCADO'],inplace=True)

In [121]:
cruce_num2.drop_duplicates(['NUMERO','IDENTIFICACION_DEUDOR'], keep='first',inplace=True)

In [122]:
cruce_num2.head()

,IDENTIFICACION_DEUDOR,NRO_IDENTIFICACION_CLIENTE,NOMBRE,SUBCAMPANAPORCLIENTE,CARTERA,TIPO,FECHA_COACTIVA,SALDO,DIAS_MORAmin,DIAS_MORAmax,...,CALL_MES,CONT_MES,CALL_MESMAS,CONT_MESMAS,DEMO,ESTADO,DEFINICION,NUMERO,NUEVO,dueno
0,00042448K,00042448K,ARAGON JUAREZ SERGIO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,94,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000559797,1000559797,RECALDE ESPINOSA ALFONSO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,189.7,36,451,...,ACUERDO DE PAGO,CLIENTE,CONTESTA IVR,TITULAR,994552348,CORRECTO,CELULAR,0994552348,NO,NaN
2,1000559797,1000559797,RECALDE ESPINOSA ALFONSO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,189.7,36,451,...,ACUERDO DE PAGO,CLIENTE,CONTESTA IVR,TITULAR,997209664,CORRECTO,CELULAR,0997209664,NO,TITULAR
3,1000580975,1000580975,VACA JERVIS SONIA,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,135,135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001296837,1001296837,BENITEZ POZO IGNACIO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,140,316,...,NO CONTESTA,NO CONTACTO,CONTESTA IVR,TITULAR,959995095,CORRECTO,CELULAR,0959995095,NO,NaN


In [123]:
cruce_uno = pd.merge(cruce_num2,gest_ase_porID_DEM,on=['NRO_IDENTIFICACION_CLIENTE','NUMERO'],how='left')
cruce_dos = pd.merge(cruce_uno,gest_mas_porID_DEM,on=['NRO_IDENTIFICACION_CLIENTE','NUMERO'],how='left')

In [124]:
cruce_dos.head()

,IDENTIFICACION_DEUDOR,NRO_IDENTIFICACION_CLIENTE,NOMBRE,SUBCAMPANAPORCLIENTE,CARTERA,TIPO,FECHA_COACTIVA,SALDO,DIAS_MORAmin,DIAS_MORAmax,...,DEMO,ESTADO,DEFINICION,NUMERO,NUEVO,dueno,CALL_MESTEL,CONT_MESTEL,CALL_MESMASTEL,CONT_MESMASTEL
0,00042448K,00042448K,ARAGON JUAREZ SERGIO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,94,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000559797,1000559797,RECALDE ESPINOSA ALFONSO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,189.7,36,451,...,994552348,CORRECTO,CELULAR,0994552348,NO,NaN,NO CONTESTA,NO CONTACTO,CONTESTA IVR,TITULAR
2,1000559797,1000559797,RECALDE ESPINOSA ALFONSO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,189.7,36,451,...,997209664,CORRECTO,CELULAR,0997209664,NO,TITULAR,ACUERDO DE PAGO,CLIENTE,CONTESTA IVR,TITULAR
3,1000580975,1000580975,VACA JERVIS SONIA,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,135,135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001296837,1001296837,BENITEZ POZO IGNACIO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,140,316,...,959995095,CORRECTO,CELULAR,0959995095,NO,NaN,NO CONTESTA,NO CONTACTO,CONTESTA IVR,TITULAR


In [125]:
asignacion_actual_dem['ASIGNADO']='SI'
asig_demo=asignacion_actual_dem[['IDENTIFICACION_DEUDOR','TELEFONO','ASIGNADO']]

In [126]:
asig_demo = asig_demo.rename(columns={'TELEFONO':'DEMO'})

In [127]:
verificacion = pd.merge(cruce_dos,asig_demo,on=['IDENTIFICACION_DEUDOR','DEMO'],how='left')

In [128]:
verificacion.drop_duplicates(['IDENTIFICACION_DEUDOR','DEMO'], keep='first',inplace=True)

In [129]:
verificacion.columns

Index(['IDENTIFICACION_DEUDOR', 'NRO_IDENTIFICACION_CLIENTE', 'NOMBRE',
       'SUBCAMPANAPORCLIENTE', 'CARTERA', 'TIPO', 'FECHA_COACTIVA', 'SALDO',
       'DIAS_MORAmin', 'DIAS_MORAmax', 'U_PLACA', 'GENERACION_CONVENIO',
       'MORA', 'PRIORIDAD', 'CAMPAÑA', 'BASE_FOCALIZADA', 'CUOTA_VENCIDA',
       'CALL_MES', 'CONT_MES', 'CALL_MESMAS', 'CONT_MESMAS', 'DEMO', 'ESTADO',
       'DEFINICION', 'NUMERO', 'NUEVO', 'dueno', 'CALL_MESTEL', 'CONT_MESTEL',
       'CALL_MESMASTEL', 'CONT_MESMASTEL', 'ASIGNADO'],
      dtype='object')

In [130]:
verificacion = verificacion.rename(columns={'NRO_IDENTIFICACION_CLIENTE':'CED'})

In [131]:
def order(frame,var):
    if type(var) is str:
        var = [var] #let the command take a string or list
    varlist =[w for w in frame.columns if w not in var]
    frame = frame[var+varlist]
    return frame

In [132]:
ordenColumn = ['NUMERO','NOMBRE','CED','SALDO','SUBCAMPANAPORCLIENTE','FECHA_COACTIVA','CARTERA','TIPO','DIAS_MORAmin','DIAS_MORAmax','U_PLACA','CAMPAÑA']

In [133]:
verificacion1 = order(verificacion,ordenColumn)

In [134]:
nuevo_archivo='BASE ARREGLADA ENERO.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
verificacion1.to_excel(writer,sheet_name='BASE_TOTAL')
worksheet = writer.sheets['BASE_TOTAL']
writer.save()